In [1]:
import os
import numpy as np
import scipy.io as sio
import pywt
from openmovement.load import CwaData

In [2]:
def load_cwa(filename):
    with CwaData(filename, include_gyro=True, include_temperature=False) as cwa_data:
        samples = cwa_data.get_samples()
        samples = samples.set_index('time')
        samples = samples.astype('float32')
        samples = samples.resample('500ms').sum() # Sub-sampled data
    return samples

In [3]:
def window_data(X,w_l=64):
    X = np.transpose(X) 
    nt, nf = X.shape
    nb = 2*nt//w_l-1
    X_w = np.zeros((nb,w_l,nf),dtype=np.float32)
    for k in range(nb):
        X_w[k,:,:] = X[k*(w_l//2):(k+1)*(w_l//2)+(w_l//2),:]
    return np.expand_dims(X_w, axis=-1) # shape: (nb,w_l,nf,1)

In [4]:
def load_data(dir_dataset, pat_file_names):#, tf_writer):
    """
    Load data and creates a dictionary.

    Parameters:
    dir_dataset: directory of dataset files.

    Returns:
    Dictionary containing all files.

    """
    ini_flag = False
    for p in pat_file_names:
        print('Patient No.',p)
        for cwa_f in os.listdir(os.path.join(dir_dataset, p, 'IMU')):
            print('  Analyzing file',cwa_f)
            cwt_flag = False
            id_num = cwa_f.split('_')[1]
            file_name = os.path.join(dir_dataset, p, 'IMU', cwa_f)
            df = load_cwa(file_name)
            sensor_idxs = list(df.keys())
            for sens_idx in sensor_idxs:
                print('    Data from sensor',sens_idx)
                if sens_idx.split('_')[0] == 'gyro':
                    scales = np.arange(1,641,10)
                else:
                    scales = np.arange(1,1025,16)
                # apply  PyWavelets continuous wavelet transfromation function
                coeffs, freqs = pywt.cwt(df[sens_idx], scales, wavelet = 'mexh')
                coeffs_w = window_data(coeffs)
                if not(cwt_flag):
                    X_file = coeffs_w
                    cwt_flag = True
                else:
                    X_file = np.concatenate((X_file,coeffs_w),axis=-1)
            if not(ini_flag):
                X = X_file
                y = [cwa_f.split('.')[0] for _ in range(X_file.shape[0])]
                ini_flag = True
            else:
                X = np.concatenate((X,X_file), axis=0)
                y += [cwa_f.split('.')[0] for _ in range(X_file.shape[0])]
    return X, y

## Create TFRecord file

In [ ]:
def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [5]:
file_dir = '../Data/'
pat_filenames = ['03','14']

recordPath = "tfrecord/"
# writer = tf.io.TFRecordWriter(recordPath + 'TFR-IMUdata')

X,y = load_data(file_dir, pat_filenames)


Patient No. 03
  Analyzing file 00_506_1.CWA
    Data from sensor accel_x
    Data from sensor accel_y
    Data from sensor accel_z
    Data from sensor gyro_x
    Data from sensor gyro_y
    Data from sensor gyro_z
Patient No. 14
  Analyzing file 01_543_3.CWA
    Data from sensor accel_x
    Data from sensor accel_y
    Data from sensor accel_z
    Data from sensor gyro_x
    Data from sensor gyro_y
    Data from sensor gyro_z


In [6]:
X.shape

(67568, 64, 64, 6)

In [8]:
len(y)

67568

In [ ]:
    
    for file_name in os.listdir(subj_path):
        if '2D_' in file_name:
            print('\t' + 'Processing ' + file_name)
            img_path = os.path.join(subj_path, file_name)
            img_path = os.path.join(img_path, os.listdir(img_path)[0])
            img_mat = sio.loadmat(img_path)
            acq = img_mat['imDataParams'][0,0][0].astype('complex64')
            TE = img_mat['imDataParams'][0,0][1].astype('float32')
            acq_real = np.real(acq)
            acq_imag = np.imag(acq)
            example = tf.train.Example(features=tf.train.Features(feature={
                'acq_real': _float_feature(acq_real.flatten()),
                'acq_imag': _float_feature(acq_imag.flatten()),
                'te': _float_feature(TE.flatten()),
                'height': _int64_feature(acq.shape[0]),
                'width': _int64_feature(acq.shape[1]),
                'num_slices': _int64_feature(acq.shape[2]),
                'num_echoes': _int64_feature(acq.shape[4])}))
            writer.write(example.SerializeToString())
        if 'results_' in file_name:
            print('\t' + 'Processing ' + file_name)
            out_path = os.path.join(subj_path, file_name)
            out_path = os.path.join(out_path, os.listdir(out_path)[0])
            out_mat = sio.loadmat(out_path)
            wf = out_mat['R'].astype('complex64')
            wf_real = np.real(wf)
            wf_imag = np.imag(wf)
            r2 = out_mat['R2'].astype('float32')
            fm = out_mat['P'].astype('float32')
            example = tf.train.Example(features=tf.train.Features(feature={
                "wf_real": _float_feature(wf_real.flatten()),
                "wf_imag": _float_feature(wf_imag.flatten()),
                "r2": _float_feature(r2.flatten()),
                "fm": _float_feature(fm.flatten()),
                'height': _int64_feature(wf.shape[0]),
                'width': _int64_feature(wf.shape[1]),
                'num_slices': _int64_feature(wf.shape[2]),
                'num_specie': _int64_feature(wf.shape[3])}))
            writer.write(example.SerializeToString())
writer.close()

## Load images from TFRecord

In [2]:
bytes('asda','utf-8')

b'asda'